In [1]:
import tensorflow as tf

In [2]:
# More imports
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, \
  Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

from sklearn.utils import shuffle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random

In [3]:
df = pd.read_csv('./BX-Book-Ratings.csv', delimiter='";"')
books_df = pd.read_csv('./BX-Books.csv', delimiter='";"')

C:\Users\danbl\AppData\Local\Temp\ipykernel_32976\865957284.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('./BX-Book-Ratings.csv', delimiter='";"')
C:\Users\danbl\AppData\Local\Temp\ipykernel_32976\865957284.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  books_df = pd.read_csv('./BX-Books.csv', delimiter='";"')


In [4]:
df.head()

,"""User-ID",ISBN,"Book-Rating"""
0,"""276725",034545104X,"0"""
1,"""276726",0155061224,"5"""
2,"""276727",0446520802,"0"""
3,"""276729",052165615X,"3"""
4,"""276729",0521795028,"6"""


In [5]:
df = df.rename(columns={'"User-ID': 'user_id', 'Book-Rating"': 'book_rating'})

In [6]:
df.head()

,user_id,ISBN,book_rating
0,"""276725",034545104X,"0"""
1,"""276726",0155061224,"5"""
2,"""276727",0446520802,"0"""
3,"""276729",052165615X,"3"""
4,"""276729",0521795028,"6"""


In [7]:
df['user_id'] = df['user_id'].str[1:]
df['book_rating'] = df['book_rating'].str[:-1]

In [8]:
df.head()

,user_id,ISBN,book_rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [9]:
books_df.head()

,"""ISBN",Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,"Image-URL-L"""
0,"""0195153448",Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,"""0002005018",Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,"""0060973129",Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,"""0374157065",Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,"""0393045218",The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [10]:
books_df = books_df.rename(columns={'"ISBN': 'ISBN', 'Image-URL-L"': 'Image-URL-L'})

In [11]:
books_df['ISBN'] = books_df['ISBN'].str[1:]
books_df['Image-URL-L'] = books_df['Image-URL-L'].str[:-1]

In [12]:
books_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [13]:
df['user_id'].values

array(['276725', '276726', '276727', ..., '276709', '276721', '276723'],
      dtype=object)

In [14]:
df['ISBN'].values

array(['034545104X', '0155061224', '0446520802', ..., '0515107662',
       '0590442449', '05162443314'], dtype=object)

In [15]:
df['user_id'] = pd.Categorical(df['user_id'])
df['new_user_id'] = df['user_id'].cat.codes

In [16]:
df['ISBN'] = pd.Categorical(df['ISBN'])
df['new_book_id'] = df['ISBN'].cat.codes

In [17]:
df['book_rating'] = pd.to_numeric(df['book_rating'], errors='coerce')

In [18]:
user_ids = df['new_user_id'].values
book_ids = df['new_book_id'].values
ratings = df['book_rating'].values

In [19]:
# Get number of users and number of movies
N = len(set(user_ids))
M = len(set(book_ids))


In [20]:
N

105283

In [21]:
M

340556

In [22]:
# split the data
user_ids, book_ids, ratings = shuffle(user_ids, book_ids, ratings)
Ntrain = int(0.8 * len(ratings))
train_user = user_ids[:Ntrain]
train_book = book_ids[:Ntrain]
train_ratings = ratings[:Ntrain]



In [23]:
test_user = user_ids[Ntrain:]
test_book = book_ids[Ntrain:]
test_ratings = ratings[Ntrain:]


In [24]:
ratings

array([ 0, 10,  0, ...,  0,  0,  0], dtype=int64)

In [25]:

# center the ratings
avg_rating = train_ratings.mean()
train_ratings = train_ratings - avg_rating
test_ratings = test_ratings - avg_rating

In [26]:
array = [128, 256, 512, 1024, 2048]
learning_rates = [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001]

models = {}
results = {}
# Make a neural network

while len(results) < 10:
  # Set variables
  K = np.random.randint(1, 10)
  # K = 10
  relu1 = random.choice(array)
  relu2 = random.choice(array)
  batch_size = random.choice(array)
  learning_rate = random.choice(learning_rates)
  # learning_rate = 0.001
  # learning_rate = 0.000001
  momentum = round(np.random.random(), 1)
  # momentum = 0.9
  epochs = np.random.randint(10, 500)
  # epochs = 25

  print([K, relu1, batch_size, learning_rate, momentum, epochs])
  # print([K, relu1, relu2, batch_size, learning_rate, momentum, epochs])

  # User input
  u = Input(shape=(1,))

  # Movie input
  b = Input(shape=(1,))

  # User embedding
  u_emb = Embedding(N, K)(u) # output is (num_samples, 1, K)

  # Book embedding
  b_emb = Embedding(M, K)(b) # output is (num_samples, 1, K)

  # Flatten both embeddings
  u_emb = Flatten()(u_emb) # now it's (num_samples, K)
  b_emb = Flatten()(b_emb) # now it's (num_samples, K)

  # Concatenate user-movie embeddings into a feature vector
  x = Concatenate()([u_emb, b_emb]) # now it's (num_samples, 2K)

  # Now that we have a feature vector, it's just a regular ANN
  x = Dense(relu1, activation='relu')(x)
  # x = Dropout(0.2)(x)
  # x = Dense(relu2, activation='relu')(x)
  x = Dense(1)(x)

  # Build the model and compile
  model = Model(inputs=[u, b], outputs=x)
  model.compile(
    loss='mse',
    optimizer=SGD(learning_rate=learning_rate, momentum=momentum),
    # optimizer=Adam(learning_rate=learning_rate)
  )

  r = model.fit(
    x=[train_user, train_book],
    y=train_ratings,
    epochs=epochs,
    batch_size=batch_size,
    verbose=0,
    validation_data=([test_user, test_book], test_ratings),
  )

  print(f"loss[0]: {r.history['loss'][0]}")
  print(f"loss[-1]: {r.history['loss'][-1]}")
  print(f"val_loss[0]: {r.history['val_loss'][0]}")
  print(f"val_loss[-1]: {r.history['val_loss'][-1]}")

  if r.history['val_loss'][-1] < 1:
    results[[K, relu1, batch_size, learning_rate, momentum, epochs]] = r.history['loss'][-1]
    models[[K, relu1, batch_size, learning_rate, momentum, epochs]] = model#
    # results[[K, relu1, relu2, batch_size, learning_rate, momentum, epochs]] = r.history['loss'][-1]
    # models[[K, relu1, relu2, batch_size, learning_rate, momentum, epochs]] = model
  
print(results)

[3, 512, 512, 0.0001, 0.5, 208]
loss[0]: 14.849335670471191
loss[-1]: 10.55537223815918
val_loss[0]: 14.873651504516602
val_loss[-1]: 11.454984664916992
[8, 2048, 512, 0.0001, 1.0, 326]
loss[0]: 14.428133964538574
loss[-1]: nan
val_loss[0]: 13.610311508178711
val_loss[-1]: nan
[2, 128, 1024, 0.001, 0.5, 452]
loss[0]: 14.84839153289795
loss[-1]: 5.823876857757568
val_loss[0]: 14.869682312011719
val_loss[-1]: 15.047636032104492
[8, 128, 2048, 1e-06, 0.6, 80]
loss[0]: 14.850330352783203
loss[-1]: 14.84999942779541
val_loss[0]: 14.874556541442871
val_loss[-1]: 14.874334335327148
[8, 2048, 256, 0.1, 0.2, 199]
loss[0]: 11.925027847290039
loss[-1]: 1.1293847560882568
val_loss[0]: 11.61359691619873
val_loss[-1]: 20.71687889099121
[7, 1024, 2048, 1e-05, 0.7, 21]
loss[0]: 14.849588394165039
loss[-1]: 14.849056243896484
val_loss[0]: 14.873845100402832
val_loss[-1]: 14.873422622680664
[9, 512, 128, 1e-05, 0.8, 267]


KeyboardInterrupt: 

In [ ]:
# plot losses
plt.plot(r.history['loss'], label="train loss")
plt.plot(r.history['val_loss'], label="val loss")
plt.legend()
plt.show()

In [ ]:
user_id = np.random.randint(0, len(set(user_ids)))
user_id

In [ ]:
watched_movie_ids = df[df.new_user_id == user_id].new_movie_id.values

In [ ]:
watched_movie_ids

In [ ]:
potential_movie_ids = df[~df.new_movie_id.isin(watched_movie_ids)].new_movie_id.unique()

In [ ]:
# How many?
potential_movie_ids.shape

In [ ]:
# Repeat the user id this many times
user_id_to_recommend = np.full_like(potential_movie_ids, user_id)

In [ ]:
predictions = model.predict([user_id_to_recommend, potential_movie_ids])

In [ ]:
predictions.shape

In [ ]:
sort_idx = np.argsort(-predictions.flatten())

In [ ]:
top_10_movie_ids = potential_movie_ids[sort_idx[:10]]
top_10_scores = predictions[sort_idx[:10]]

for movie, score in zip(top_10_movie_ids, top_10_scores):
  movieId = df.loc[df['new_movie_id'] == movie, 'movieId'].values[0]
  movie_string = movies_df.loc[movies_df['movieId'] == movieId, 'title'].values[0]
  print("movie:", movie_string, "score:", score)